For tensor board SSH

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-05 19:13:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.237.103, 54.174.228.92, 52.55.191.55, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.237.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  14.30M  7.07MB/s    in 2.0s    

2019-05-05 19:13:28 (7.07 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import tensorflow as tf
path_to_events_file = './save/train/baseline-11/log.txt'
for e in tf.train.summary_iterator(path_to_events_file):
    for v in e.summary.value:
        if v.tag == 'loss' or v.tag == 'accuracy':
            print(v.simple_value)

DataLossError: ignored

In [0]:
!ls

args.py		 ngrok				 save
data		 ngrok-stable-linux-amd64.zip	 setup.py
environment.yml  ngrok-stable-linux-amd64.zip.1  test.py
layers.py	 ngrok-stable-linux-amd64.zip.2  train.py
models.py	 __pycache__			 util.py


In [0]:
LOG_DIR = './log.txt'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006'.format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://8aaf8bfa.ngrok.io


In [0]:
%cd ..

/content/squad


In [0]:
!python train.py -n baseline

python3: can't open file 'train.py': [Errno 2] No such file or directory


In [0]:
import random, string, urllib.request, json, getpass

#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')

#Print root password
print(f'Root password: {password}')

Copy authtoken from https://dashboard.ngrok.com/auth
··········
SSH command: ssh -p19666 root@0.tcp.ngrok.io
Root password: FW5fvwAQPljSuphP6LJv


In [0]:
#user ssh: -p19666 root@0.tcp.ngrok.io
# pass ssh: FW5fvwAQPljSuphP6LJv

In [0]:
from google.colab import drive, files
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!git clone https://github.com/chrischute/squad.git

Cloning into 'squad'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17
Unpacking objects: 100% (17/17), done.


change directory to squad
> %cd squad or %cd content/squad

In [0]:
!ls


sample_data  squad


In [0]:
%cd squad/data

/content/squad/data


In [0]:
# %cd squad
import sys, os
sys.path.append(os.getcwd())


In [0]:
!ls


ngrok  ngrok-stable-linux-amd64.zip  sample_data  squad


In [0]:
# %cd data
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



     |████████████████████████████████| 993kB 3.3MB/s 


download train into data directory. give gdrive id of train-v2- get shareable link has id in the link

In [0]:
import json
# %cd data
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1Gu2dm8YO8xIvUdglWuCLdQYjwpnnYU0b'
downloaded = drive.CreateFile({'id': file_id})
downloaded_content = downloaded.GetContentString()

In [0]:
downloaded_content = json.loads(downloaded_content)

In [0]:

!ls


dev-v2.0.json  test-v2.0.json  train-v2.0.json


Update train file as json into data directory (i.e. current directory)

In [0]:
filename = 'train-v2.0.json'
with open(filename, "w") as fh:
    json.dump(downloaded_content, fh)

Change directory to squad

In [0]:
%cd ..


/content/squad


In [0]:
!python setup.py

glove.840B.300d.zip: 2.18GB [01:03, 34.4MB/s]                
Unzipping GloVe word vectors...

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

Pre-processing train examples...
100% 119/119 [00:20<00:00,  6.62it/s]
36024 questions in total
Pre-processing word vectors...
2196018it [03:13, 11337.50it/s]                
44625 / 50048 tokens have corresponding word embedding vector
Pre-processing char vectors...
621 tokens have corresponding char embedding vector
Pre-processing dev examples...
100% 16/16 [00:03<00:00,  5.45it/s]
6078 questions in total
Converting train examples to indices...
36024it [00:14, 2413.89it/s]
Built 35938 / 36024 instances of features in total
Converting dev examples to indices...
6078it [00:02, 2458.03it/s]
Built 5951 / 6078 instances of features in total
Pre-processing test examples...
100% 20/20 [00:02<00:00,  9.39i

In [0]:
!pip install tensorboardX

     |████████████████████████████████| 133kB 3.3MB/s 


In [0]:
!pip install tensorboardcolab


NLL should keep decreasing in each epoch. NLL is error.

In [0]:

!python train.py -n baseline

[05.06.19 04:19:20] Args: {
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "drop_prob": 0.2,
    "ema_decay": 0.999,
    "eval_steps": 50000,
    "gpu_ids": [
        0
    ],
    "hidden_size": 100,
    "l2_wd": 0,
    "load_path": null,
    "lr": 0.5,
    "max_ans_len": 15,
    "max_checkpoints": 5,
    "max_grad_norm": 5.0,
    "maximize_metric": true,
    "metric_name": "F1",
    "name": "baseline",
    "num_epochs": 30,
    "num_visuals": 10,
    "num_workers": 4,
    "save_dir": "./save/train/baseline-01",
    "seed": 224,
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[05.06.19 04:19:20] Using random seed 224...
[05.06.19 04:19:20] Loading embeddings...
[05.06.19 04

In [0]:
%cd squad

/content/squad


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving layers.py to layers.py
User uploaded file "layers.py" with length 9133 bytes


ignore anything below 

In [0]:
%cd data

[Errno 2] No such file or directory: 'data'
/


In [0]:
!ls

bin	 datalab  gdrive  lib32  mnt   root  srv    tmp    var
boot	 dev	  home	  lib64  opt   run   swift  tools
content  etc	  lib	  media  proc  sbin  sys    usr


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a text file.
with open('train_eval.json') as f:
# uploaded = drive.CreateFile({'title': 'Sample file.txt'})
# uploaded.SetContentString('Sample upload file content')
  f.Upload()
# print('Uploaded file with ID {}'.format(uploaded.get('id')))

AttributeError: ignored

In [0]:
!python test.py -n baseline


Traceback (most recent call last):
  File "test.py", line 139, in <module>
    main(get_test_args())
  File "/content/squad/args.py", line 173, in get_test_args
    raise argparse.ArgumentError('Missing required argument --load_path')
TypeError: __init__() missing 1 required positional argument: 'message'


In [0]:
%cd squad

/content/squad
